In [ ]:
# Premonitor: hardware_drivers.py
# This file contains the functions and classes to control the physical hardware
# components connected to the Raspberry Pi.
# This code is designed to be robust, non-blocking, and configurable.
# It will replace 'mock_hardware.py' in a real deployment.

import time
import threading
import numpy as np

# --- Import hardware-specific libraries ---
# On a real Raspberry Pi, these should be installed:
# > pip install rpi_ws281x adafruit-circuitpython-neopixel adafruit-circuitpython-mcp3xxx
# > pip install pygame pyttsx3 sounddevice
try:
    import board
    import neopixel
    import pygame
    import pyttsx3
    import sounddevice as sd
    import adafruit_mcp3xxx.mcp3008 as MCP
    from adafruit_mcp3xxx.analog_in import AnalogIn
    import busio
    import digitalio
    import spidev
    HARDWARE_AVAILABLE = True
except (ImportError, NotImplementedError, FileNotFoundError):
    print("HARDWARE_DRIVERS: One or more hardware libraries not found. Running in simulation mode.")
    HARDWARE_AVAILABLE = False

# Import our custom project configuration
import config
import utils

# --- Global hardware objects ---
led_controller = None
tts_engine = None
gas_sensor_adc = None

# --- LED Controller Class (for non-blocking animations) ---
class LEDController(threading.Thread):
    """A dedicated thread to manage LED animations without blocking the main loop."""
    def __init__(self):
        super().__init__(daemon=True)
        self.pixels = None
        self.status = 'booting'
        self.lock = threading.Lock()
        self._stop_event = threading.Event()

        # State for the 'normal' scanning animation
        self.scan_pos = 0
        self.scan_dir = 1

        if HARDWARE_AVAILABLE:
            try:
                self.pixels = neopixel.NeoPixel(
                    config.LED_PIN,
                    config.LED_PIXEL_COUNT,
                    brightness=config.LED_BRIGHTNESS,
                    auto_write=False
                )
                print(" - LED strip initialized.")
            except Exception as e:
                print(f" - CRITICAL Error initializing LED strip: {e}")
                self.pixels = None

    def run(self):
        """The main loop for the LED thread, handling different animation states."""
        if not self.pixels:
            return

        cycle = 0
        while not self._stop_event.is_set():
            with self.lock:
                current_status = self.status

            # Clear pixels at the start of each animation frame
            self.pixels.fill((0, 0, 0))

            if current_status == 'critical':
                # Fast Blinking Red
                if cycle % 4 < 2:
                    self.pixels.fill((255, 0, 0))
            elif current_status == 'warning':
                # Blinking Orange
                if cycle % 10 < 5:
                    self.pixels.fill((255, 100, 0))
            elif current_status == 'normal':
                # Green "Scanning" light moving back and forth
                self.pixels[self.scan_pos] = (0, 255, 0)
                # Update position for the next frame
                if self.scan_pos + self.scan_dir >= config.LED_PIXEL_COUNT or self.scan_pos + self.scan_dir < 0:
                    self.scan_dir *= -1 # Reverse direction
                self.scan_pos += self.scan_dir
            elif current_status == 'booting':
                # Rainbow chase animation
                for i in range(config.LED_PIXEL_COUNT):
                    pixel_index = (i * 256 // config.LED_PIXEL_COUNT) + cycle
                    self.pixels[i] = self.wheel(pixel_index & 255)

            self.pixels.show()
            cycle += 1
            time.sleep(0.05) # Animation speed control

    def set_status(self, new_status):
        """Thread-safe method to change the current LED animation state."""
        with self.lock:
            self.status = new_status
            # Reset scanner position when switching to normal
            if new_status == 'normal':
                self.scan_pos = 0
                self.scan_dir = 1

    def stop(self):
        """Stops the animation thread and turns off LEDs."""
        self._stop_event.set()
        if self.pixels:
            time.sleep(0.1) # Allow thread to finish its last cycle
            self.pixels.fill((0, 0, 0))
            self.pixels.show()

    def wheel(self, pos):
        """Helper function to generate rainbow colors."""
        if pos < 85:
            return (pos * 3, 255 - pos * 3, 0)
        elif pos < 170:
            pos -= 85
            return (255 - pos * 3, 0, pos * 3)
        else:
            pos -= 170
            return (0, pos * 3, 255 - pos * 3)

# --- Initialization & Shutdown ---
def initialize_all_hardware():
    """Initializes all hardware components at the start of the application."""
    global led_controller, tts_engine, gas_sensor_adc
    if not HARDWARE_AVAILABLE:
        print("HARDWARE_DRIVERS: Skipping hardware initialization.")
        return

    print("HARDWARE_DRIVERS: Initializing components...")

    led_controller = LEDController()
    led_controller.start()

    try:
        pygame.mixer.init()
        print(" - Pygame mixer for audio playback initialized.")
    except Exception as e:
        print(f" - Error initializing Pygame mixer: {e}")

    try:
        tts_engine = pyttsx3.init()
        print(" - Text-to-Speech engine initialized.")
    except Exception as e:
        print(f" - Error initializing TTS engine: {e}")

    try:
        spi = busio.SPI(clock=config.ADC_SPI_CLOCK_PIN, MOSI=config.ADC_SPI_MOSI_PIN, MISO=config.ADC_SPI_MISO_PIN)
        cs = digitalio.DigitalInOut(config.ADC_SPI_CS_PIN)
        mcp = MCP.MCP3008(spi, cs)
        gas_sensor_adc = AnalogIn(mcp, config.GAS_SENSOR_ADC_CHANNEL)
        print(" - ADC for gas sensor initialized.")
    except Exception as e:
        print(f" - Error initializing ADC: {e}")

    print("HARDWARE_DRIVERS: Initialization complete.")
    # Show booting animation for 2 seconds then switch to normal
    time.sleep(2)
    set_led_status('normal')


def shutdown_hardware():
    """Safely shuts down hardware components."""
    print("HARDWARE_DRIVERS: Shutting down components...")
    if led_controller:
        led_controller.stop()
    if HARDWARE_AVAILABLE:
        # Import GPIO here to avoid errors if not on a Pi
        import RPi.GPIO as GPIO
        GPIO.cleanup()
    print("HARDWARE_DRIVERS: Shutdown complete.")


# --- Sensor Reading Functions (Real Implementations) ---
def read_thermal_image():
    """
    TODO: Reads from a real thermal camera (e.g., FLIR Lepton).
    This requires a specific library like 'pylepton' or similar.
    """
    if not HARDWARE_AVAILABLE:
        return np.random.rand(*config.THERMAL_MODEL_INPUT_SHAPE).astype(np.float32)
    return np.zeros(config.THERMAL_MODEL_INPUT_SHAPE, dtype=np.float32)

def read_acoustic_spectrogram():
    """
    Reads a chunk of audio from a real microphone, then converts it to a spectrogram.
    """
    if not HARDWARE_AVAILABLE:
        return np.zeros(config.ACOUSTIC_MODEL_INPUT_SHAPE, dtype=np.float32)

    try:
        duration = 3 # seconds
        audio_chunk = sd.rec(int(duration * config.SPECTROGRAM_SAMPLE_RATE), samplerate=config.SPECTROGRAM_SAMPLE_RATE, channels=1, dtype='float32')
        sd.wait()

        spectrogram = utils.audio_to_spectrogram_from_array(audio_chunk.flatten(), sr=config.SPECTROGRAM_SAMPLE_RATE)
        if spectrogram is not None:
            return tf.image.resize(spectrogram[..., np.newaxis], config.ACOUSTIC_MODEL_INPUT_SHAPE[:2]).numpy()
    except Exception as e:
        print(f"HARDWARE_DRIVERS: Error reading from microphone: {e}")

    return np.zeros(config.ACOUSTIC_MODEL_INPUT_SHAPE, dtype=np.float32)

def read_gas_sensor():
    """Reads the value from the analog gas sensor via the MCP3008 ADC."""
    if not gas_sensor_adc:
        return random.randint(200, 350)
    return gas_sensor_adc.value

# --- UI Feedback Functions ---
def set_led_status(status='normal'):
    """Public function to control the LED status via the controller."""
    if led_controller:
        led_controller.set_status(status)

def play_sound(sound_file_name):
    """Plays a sound file from the 'sounds' directory."""
    if not HARDWARE_AVAILABLE or not pygame.mixer.get_init():
        if config.DEBUG_MODE: print(f"AUDIO_DRIVER: Simulating playing sound '{sound_file_name}'")
        return

    sound_path = os.path.join(config.SOUNDS_DIR, sound_file_name)
    if os.path.exists(sound_path):
        try:
            pygame.mixer.music.load(sound_path)
            pygame.mixer.music.play()
        except Exception as e:
            print(f"AUDIO_DRIVER: Error playing sound file {sound_path}: {e}")
    else:
        print(f"AUDIO_DRIVER: Sound file not found: {sound_path}")

def speak_text(text_to_speak):
    """Uses the text-to-speech engine to speak a given string."""
    if not tts_engine:
        if config.DEBUG_MODE: print(f"TTS_DRIVER: Simulating speech: '{text_to_speak}'")
        return

    try:
        # Run in a separate thread to avoid blocking
        threading.Thread(target=tts_engine.say, args=(text_to_speak,)).start()
        threading.Thread(target=tts_engine.runAndWait).start()
    except Exception as e:
        print(f"TTS_DRIVER: Error in text-to-speech: {e}")